In [11]:
# Dependencies and config file
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
import openweathermapy as owm
from citipy import citipy
from random import randint
import numpy as np

In [49]:
city = citipy.nearest_city(22.99, 120.21)
#print(city)
city.city_name
#city.country_code

'tainan'

In [42]:
latitude_list = []
for x in range(500):
    # Randomly select 500 unique latitudes
    latitude_list.append(numpy.random.uniform(low=-180.0, high=180, size=None))
    #print(latitude_list)
    # this works but it is a very long list 
latitude_list_series = pd.Series(latitude_list)
#Then add the values to the DataFrame:

#city_df['new_col'] = latitude_list_series.values

In [43]:
longitude_list = []
for x in range(500):
    # Randomly select 500 unique longitudes
    longitude_list.append(numpy.random.uniform(low=-90, high = 90))
    #print(longitude)

# Convert list to series
longitude_list_series = pd.Series(longitude_list)

In [50]:
lat_and_long = pd.concat([latitude_list_series, longitude_list_series], axis=1)
lat_and_long.head(10)

,0,1
0,154.843382,-59.159888
1,-96.629807,12.393261
2,64.712780,4.664163
3,-169.752406,46.244465
4,-21.747498,31.766596
5,-34.333861,-19.292775
6,116.545072,-15.359043
7,150.214511,61.631034
8,-86.443870,7.173107
9,112.837336,38.275250


In [ ]:
# possibly use the zip function to create one value and then use the city instance?

In [46]:
#for i, j in lat_and_long:
    
for index, row in lat_and_long.iterrows():
    city = citipy.city_name(x, y)
    
#city
#citipy.City instance at 0x1069b6518>
#city.city_name     # Tainan, my home town
#'tainan'
#city.country_code
#'tw'                  # And the country is surely Taiwan

In [23]:
# empty dataframe is this needed?
d = {'city': [""], 'latitude': [""], 'longitude': [""]}
city_df = pd.DataFrame(data=d, index=[0,1,2,3,4,5,6,500])
city_df.head(25)
#city_data = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

,city,latitude,longitude
0,,,
1,,,
2,,,
3,,,
4,,,
5,,,
6,,,
500,,,


In [9]:
# Create settings dictionary 
#settings = {"units": "imperial", "APPID": api_key}

In [ ]:
# Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
# Perform a weather check on each of the cities using a series of successive API calls. 
# Include a print log of each city as it's being processed with the city number, city name, and requested URL.
# Save both a CSV of all data retrieved and png images for each scatter plot.

In [ ]:
#Your objective is to build a series of scatter plots to showcase the following relationships:

#* Temperature (F) vs. Latitude
#* Humidity (%) vs. Latitude
#* Cloudiness (%) vs. Latitude
#* Wind Speed (mph) vs. Latitude

In [ ]:
# Build a scatter plot for each data type
# Example from other activity
#plt.scatter(census_data_complete["Bank Count"].astype(int),
            #census_data_complete["Unemployment Rate"],
            #edgecolor="black", linewidths=1, marker="o",
            #alpha=0.8, label="Zip Codes")

# Label the graph and set axis limits
#plt.title("Unemployment Rate vs. Bank Count by Zip Code")
#plt.ylabel("Unemployment Rate (%)")
#plt.xlabel("Bank Count")
#plt.grid(True)
#plt.xlim([-2.5, 202])
#plt.ylim([-2.5, 40])

# Save the figure
#plt.savefig("UnemploymentRate_BankCount.png")

# Show plot
#plt.show()